### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,accuracy_score,\
log_loss,confusion_matrix
import math
import warnings
warnings.filterwarnings('ignore')

### Reading Data

In [2]:
df=pd.read_csv("bank.xls")

In [3]:
df.drop(['ID'],axis=1,inplace=True)

### Downsampling Data

In [4]:
one_class=df[df['Personal Loan']==1]
zero_class=df[df['Personal Loan']==0]

In [5]:
zero_downsample = resample(zero_class,
             replace=True,
             n_samples=len(one_class),
             random_state=42)

In [6]:
data=pd.concat([zero_downsample,one_class])

In [7]:
X=data.drop(['Personal Loan'],axis=1).values
Y=data['Personal Loan'].values

### Splitting Data

In [8]:
X_train,X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [9]:
X_train.shape

(768, 12)

In [10]:
sc=StandardScaler()

In [11]:
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

### Genetic Algorithm for Weight Optimisation

In [12]:
class geneticNN:
    
    def __init__(self, X_train, y_train, hidden_layer_size=10, generations=20, chromosomes=30,
                 mating_parents=10, crossover_rate=0.85, mutation_rate=0.4):
        self.input_layer_size = X_train.shape[1]
        self.hidden_layer_size = hidden_layer_size
        self.output_layer_size = 1
        self.generations = generations
        self.chromosomes = chromosomes
        self.mating_parents = mating_parents
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.equation_inputs = X_train
        self.population = np.random.rand(self.chromosomes, self.get_genes())
        self.population = 2 * self.population - 1
        self.y_train = y_train
        self.best_weights=None
        
    
    def get_genes(self):
        return self.input_layer_size * self.hidden_layer_size + self.hidden_layer_size + self.hidden_layer_size * \
               self.output_layer_size + self.output_layer_size
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def tanh(self, x):
        return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))

    def forward(self, inputs, weights):
        weight1 = np.reshape(weights[:self.hidden_layer_size * self.input_layer_size],
                              (self.hidden_layer_size, self.input_layer_size))
        bias1 = np.reshape(weights[self.hidden_layer_size * self.input_layer_size:self.hidden_layer_size *
                                   self.input_layer_size + self.hidden_layer_size], (self.output_layer_size,self.hidden_layer_size))
        weight2 = np.reshape(weights[self.hidden_layer_size * self.input_layer_size + self.hidden_layer_size:-1],
                              (self.output_layer_size, self.hidden_layer_size))
        bias2 = np.reshape(weights[-1], (self.output_layer_size, 1))

        A0 = inputs
        Z1 = A0.dot(weight1.T)+bias1
        A1 = self.sigmoid(Z1)
        Z2 = A1.dot(weight2.T)+bias2
        A2 = self.sigmoid(Z2)
        y_pred = np.round(A2)

        return y_pred

    def fitness_function(self, inputs, population, y_train):
        fitness = []
        for weights in population:
            y_pred = self.forward(inputs, weights)
            acc = accuracy_score(y_train, y_pred)

            fitness.append(acc)

        return fitness

    def select_parents(self, population, fitness, mating_parents):
        parents = np.zeros((mating_parents, population.shape[1]))
        for parent_num in range(mating_parents):
            max_fitness_index = np.where(fitness == np.max(fitness))
            max_fitness_index = max_fitness_index[0][0]
            parents[parent_num, :] = population[max_fitness_index, :]
            fitness[max_fitness_index] = -999999
        return parents

    def crossover(self, parents, offspring_size):
        offspring = np.empty(offspring_size)

        crossover_point = np.uint8(offspring_size[1] / 2)

        for k in range(offspring_size[0]):
            parent1_index = k % parents.shape[0]
            parent2_index = (k + 1) % parents.shape[0]

            # Generate a random number between 0 and 1
            r = np.random.rand()

            # If random number is less than crossover rate, perform crossover
            if r < self.crossover_rate:
                offspring[k, 0:crossover_point] = parents[parent1_index, 0:crossover_point]
                offspring[k, crossover_point:] = parents[parent2_index, crossover_point:]
            else:
                offspring[k, :] = parents[parent1_index, :]

        return offspring

    def mutation(self, offspring_crossover):
        
        num_mutations = int(offspring_crossover.shape[0] * self.mutation_rate)
    
        mutation_indices = np.random.choice(offspring_crossover.shape[0], num_mutations, replace=False)

        for i in mutation_indices:
            offspring_crossover[i, 4] += np.random.uniform(-1, 1)
        
        return offspring_crossover

    def generate_new_population(self, parents, offspring_size):
        offspring_crossover = self.crossover(parents, offspring_size)
        offspring_mutation = self.mutation(offspring_crossover)
        return offspring_mutation

    def run(self):
        for generation in range(self.generations):
            
            fitness = self.fitness_function(self.equation_inputs, self.population, self.y_train)
            
            parents = self.select_parents(self.population, fitness, self.mating_parents)

            offspring_size = (self.chromosomes - parents.shape[0], self.get_genes())

            offspring_mutation = self.generate_new_population(parents, offspring_size)

            self.population[0:parents.shape[0], :] = parents
            self.population[parents.shape[0]:, :] = offspring_mutation

            print("Generation:", generation, "Accuracy:", np.max(fitness))

        self.best_weights = self.population[np.where(fitness == np.max(fitness))][0]
        #print(self.population[np.where(fitness == np.max(fitness))][0])
        #return self.best_weights.shape
        
    def predict(self,X_test):
        prediction=self.forward(X_test,self.best_weights)
        return prediction
    
    def reports(self,X_test,y_test):
        prediction=self.forward(X_test,self.best_weights)
        
        print(f"ACCURACY : {accuracy_score(y_test,prediction)}\n\n\
CLASSIFICATION REPORT :\n {classification_report(y_test,prediction)}")


In [13]:
geneticnn=geneticNN(X_train,y_train,
    hidden_layer_size=10,
    generations=25,
    chromosomes=40,
    mating_parents=9,
    crossover_rate=0.80,
    mutation_rate=0.4)

In [14]:
geneticnn.run()

Generation: 0 Accuracy: 0.5182291666666666
Generation: 1 Accuracy: 0.6145833333333334
Generation: 2 Accuracy: 0.734375
Generation: 3 Accuracy: 0.7356770833333334
Generation: 4 Accuracy: 0.7421875
Generation: 5 Accuracy: 0.7421875
Generation: 6 Accuracy: 0.7434895833333334
Generation: 7 Accuracy: 0.7434895833333334
Generation: 8 Accuracy: 0.7434895833333334
Generation: 9 Accuracy: 0.7434895833333334
Generation: 10 Accuracy: 0.7434895833333334
Generation: 11 Accuracy: 0.7434895833333334
Generation: 12 Accuracy: 0.7434895833333334
Generation: 13 Accuracy: 0.7434895833333334
Generation: 14 Accuracy: 0.7434895833333334
Generation: 15 Accuracy: 0.7434895833333334
Generation: 16 Accuracy: 0.7434895833333334
Generation: 17 Accuracy: 0.7434895833333334
Generation: 18 Accuracy: 0.7434895833333334
Generation: 19 Accuracy: 0.7434895833333334
Generation: 20 Accuracy: 0.7434895833333334
Generation: 21 Accuracy: 0.7434895833333334
Generation: 22 Accuracy: 0.7434895833333334
Generation: 23 Accuracy: 0

In [15]:
pred=geneticnn.predict(X_test)

In [16]:
geneticnn.reports(X_test,y_test)

ACCURACY : 0.7864583333333334

CLASSIFICATION REPORT :
               precision    recall  f1-score   support

           0       0.87      0.62      0.72        86
           1       0.75      0.92      0.83       106

    accuracy                           0.79       192
   macro avg       0.81      0.77      0.77       192
weighted avg       0.80      0.79      0.78       192

